The data preprocessing phase.
---

After data collection, we cleaned the data since our data set was quite imbalance and had a lot of features. We dropped inull values and rrelevant features such as ID, Case, Number, FBI code, Updated On, ICUR, X and Y Coordinates, Location and Description. There were 34 distinct types of crimes and we dropped those crimes we thought were not very significant. Similar types of crime such as sexual assault, prostitution and sex offenses were mierged together. Hence after preprocessing the primary crime types dropped from 35 to 18. A new csv file was created after pre-processing for analysis.

Download the dataset from link  https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2

and place in dataset folder.



In [2]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/11/16 01:06:30 WARN Utils: Your hostname, praveens-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
23/11/16 01:06:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/16 01:06:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/16 01:06:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv
from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when
import pyspark.sql.functions as F

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

23/11/16 01:06:43 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [8]:
#change RAM allocation, current = 20GB
#change Core allocation, current = 3 Cores

spark = SparkSession.builder.appName("Project - Chicago crime").config("spark.some.config.option", "some-value").config("spark.driver.memory", "16g").config("spark.driver.cores", '3').getOrCreate()



23/11/16 01:08:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
crimes_schema = StructType([StructField("ID", IntegerType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", IntegerType(), True),
                            StructField("District", IntegerType(), True),
                            StructField("Ward", IntegerType(), True),
                            StructField("Community Area", IntegerType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", IntegerType(), True),
                            StructField("Y Coordinate", IntegerType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", StringType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [10]:
data = spark.read.csv('Dataset/Crimes_-_2001_to_Present.csv',header = True,schema = crimes_schema)

In [11]:
data.count()
print(len(data.columns))

22


In [12]:
data.head(5)

[Row(ID=5741943, Case Number='HN549294', Date='08/25/2007 09:22:18 AM', Block='074XX N ROGERS AVE', IUCR='0560', Primary Type='ASSAULT', Description='SIMPLE', Location Description='OTHER', Arrest=False, Domestic=False, Beat=2422, District=24, Ward=49, Community Area=1, FBI Code='08A', X Coordinate=None, Y Coordinate=None, Year=2007, Updated On='08/17/2015 03:03:40 PM', Latitude=None, Longitude=None, Location=None),
 Row(ID=1930689, Case Number='HH109118', Date='01/05/2002 09:24:00 PM', Block='007XX E 103 ST', IUCR='0820', Primary Type='THEFT', Description='$500 AND UNDER', Location Description='GAS STATION', Arrest=True, Domestic=False, Beat=512, District=5, Ward=None, Community Area=None, FBI Code='06', X Coordinate=None, Y Coordinate=None, Year=2002, Updated On='02/04/2016 06:33:39 AM', Latitude=None, Longitude=None, Location=None),
 Row(ID=13203321, Case Number='JG415333', Date='09/06/2023 05:00:00 PM', Block='002XX N Wells st', IUCR='1320', Primary Type='CRIMINAL DAMAGE', Descripti

# Data preprocessing - Spark

**Working on data starting 2010.**

In [13]:

# dataset = data.filter((data['Year'] >2015 & (data['Year'] !=2020))

dataset = data.filter((data['Year'] > 2009))

In [14]:
#dropping columns which non-relative 

dataset = dataset.drop('ID')
dataset = dataset.drop('Case Number')
dataset = dataset.drop('FBI Code')
dataset = dataset.drop('Updated On')
dataset = dataset.drop('IUCR')# Could be useful - four digit codes that law enforcement agencies use to classify criminal incidents when taking individual reports.
dataset = dataset.drop('X Coordinate')
dataset = dataset.drop('Y Coordinate')
dataset = dataset.drop('Location')
dataset = dataset.drop('Description')
dataset = dataset.drop('Domestic')

**Preprocessing Primary types**

In [15]:
primaryCount = dataset.groupby('Primary Type').count()
primaryCount.orderBy('count', ascending=False).show()

+--------------------+------+
|        Primary Type| count|
+--------------------+------+
|               THEFT|856662|
|             BATTERY|696965|
|     CRIMINAL DAMAGE|418588|
|           NARCOTICS|280202|
|             ASSAULT|271578|
|       OTHER OFFENSE|235224|
|  DECEPTIVE PRACTICE|229405|
|            BURGLARY|199495|
| MOTOR VEHICLE THEFT|196296|
|             ROBBERY|148516|
|   CRIMINAL TRESPASS| 91011|
|   WEAPONS VIOLATION| 74872|
|OFFENSE INVOLVING...| 31919|
|PUBLIC PEACE VIOL...| 27438|
|         SEX OFFENSE| 15615|
|        PROSTITUTION| 15500|
|INTERFERENCE WITH...| 13790|
| CRIM SEXUAL ASSAULT| 13488|
|            HOMICIDE|  8201|
|CRIMINAL SEXUAL A...|  7390|
+--------------------+------+
only showing top 20 rows



In [16]:
print('Distinct primary types '+ str(primaryCount.distinct().count()))

Distinct primary types 35


**Drop crimes which are not so significant**

In [17]:
dataset = dataset.filter((dataset['Primary Type']!='STALKING')&
                             (dataset['Primary Type']!='OBSCENITY')&
                             (dataset['Primary Type']!='NON-CRIMINAL (SUBJECT SPECIFIED)')&
                             (dataset['Primary Type']!='GAMBLING')&
                             (dataset['Primary Type']!='NON - CRIMINAL')&
                             (dataset['Primary Type']!='LIQUOR LAW VIOLATION')&
                             (dataset['Primary Type']!='PUBLIC INDECENCY')&
                             (dataset['Primary Type']!='HUMAN TRAFFICKING')&
                             (dataset['Primary Type']!='INTIMIDATION')&
                             (dataset['Primary Type']!='CONCEALED CARRY LICENSE VIOLATION')&
                             (dataset['Primary Type']!='NON-CRIMINAL')&
                             (dataset['Primary Type']!='OTHER NARCOTIC VIOLATION'))

**Merging similar crime types**

In [18]:
dataset = dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'CRIM SEXUAL ASSAULT') | \
                                    (dataset["Primary Type"] == 'PROSTITUTION') ,'SEX OFFENSE').otherwise(dataset['Primary Type']))
dataset=dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'WEAPONS VIOLATION') | \
                                    (dataset["Primary Type"] == 'INTERFERENCE WITH PUBLIC OFFICER') ,'PUBLIC PEACE VIOLATION').otherwise(dataset['Primary Type']))
dataset=dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'ARSON') ,'CRIMINAL TRESPASS').otherwise(dataset['Primary Type']))

dataset.show(5)

+--------------------+--------------------+--------------------+--------------------+------+----+--------+----+--------------+----+------------+-------------+
|                Date|               Block|        Primary Type|Location Description|Arrest|Beat|District|Ward|Community Area|Year|    Latitude|    Longitude|
+--------------------+--------------------+--------------------+--------------------+------+----+--------+----+--------------+----+------------+-------------+
|09/06/2023 05:00:...|    002XX N Wells st|     CRIMINAL DAMAGE|PARKING LOT / GAR...| false| 122|       1|  42|            32|2023|41.886018055|-87.633937881|
|08/31/2023 12:00:...|023XX W JACKSON BLVD|  DECEPTIVE PRACTICE|              STREET| false|1225|      12|  27|            28|2023|41.877565108| -87.68479102|
|07/24/2023 09:45:...|073XX S JEFFERY BLVD|CRIMINAL SEXUAL A...|           APARTMENT| false| 333|       3|   7|            43|2023|  41.7619185|-87.576209245|
|08/27/2023 07:00:...|034XX N LAWNDALE AVE|   

In [19]:
primaryCount = dataset.groupby('Primary Type').count()
primaryCount.orderBy('count', ascending=False).show()

+--------------------+------+
|        Primary Type| count|
+--------------------+------+
|               THEFT|856662|
|             BATTERY|696965|
|     CRIMINAL DAMAGE|418588|
|           NARCOTICS|280202|
|             ASSAULT|271578|
|       OTHER OFFENSE|235224|
|  DECEPTIVE PRACTICE|229405|
|            BURGLARY|199495|
| MOTOR VEHICLE THEFT|196296|
|             ROBBERY|148516|
|PUBLIC PEACE VIOL...|116100|
|   CRIMINAL TRESPASS| 97393|
|         SEX OFFENSE| 44603|
|OFFENSE INVOLVING...| 31919|
|            HOMICIDE|  8201|
|CRIMINAL SEXUAL A...|  7390|
|          KIDNAPPING|  2652|
|           RITUALISM|     1|
+--------------------+------+



In [20]:
print('Distinct primary types after preprocessing '+ str(primaryCount.distinct().count()))

Distinct primary types after preprocessing 18


**Extract Day, Month, Week from Date Column**

In [21]:
dataset = dataset.withColumn("Day", F.split(dataset.Date, " ")[0])
dataset = dataset.withColumn("Day", F.to_date(dataset.Day, "MM/dd/yyyy"))
dataset = dataset.withColumn("Month", F.month(dataset.Day))
dataset = dataset.withColumn("WeekDay", F.dayofweek(dataset.Day))
dataset = dataset.withColumn("Year", F.year(dataset.Day))
dataset = dataset.na.drop()

#dataset = dataset.drop('Day')
#dataset = dataset.drop('Date')
dataset.head(5)

[Row(Date='09/06/2023 05:00:00 PM', Block='002XX N Wells st', Primary Type='CRIMINAL DAMAGE', Location Description='PARKING LOT / GARAGE (NON RESIDENTIAL)', Arrest=False, Beat=122, District=1, Ward=42, Community Area=32, Year=2023, Latitude=41.886018055, Longitude=-87.633937881, Day=datetime.date(2023, 9, 6), Month=9, WeekDay=4),
 Row(Date='08/31/2023 12:00:00 PM', Block='023XX W JACKSON BLVD', Primary Type='DECEPTIVE PRACTICE', Location Description='STREET', Arrest=False, Beat=1225, District=12, Ward=27, Community Area=28, Year=2023, Latitude=41.877565108, Longitude=-87.68479102, Day=datetime.date(2023, 8, 31), Month=8, WeekDay=5),
 Row(Date='07/24/2023 09:45:00 PM', Block='073XX S JEFFERY BLVD', Primary Type='CRIMINAL SEXUAL ASSAULT', Location Description='APARTMENT', Arrest=False, Beat=333, District=3, Ward=7, Community Area=43, Year=2023, Latitude=41.7619185, Longitude=-87.576209245, Day=datetime.date(2023, 7, 24), Month=7, WeekDay=2),
 Row(Date='08/27/2023 07:00:00 AM', Block='034

Ignore Latitude and Longitude outside Chichago

In [22]:
dataset=dataset\
        .filter((dataset["Latitude"] < 45)
             & (dataset["Latitude"] > 40)
             & (dataset["Longitude"] < -85)
             & (dataset["Longitude"] > -90))

**Dropping Null values in the data.**

In [23]:
dataset.head(5)

[Row(Date='09/06/2023 05:00:00 PM', Block='002XX N Wells st', Primary Type='CRIMINAL DAMAGE', Location Description='PARKING LOT / GARAGE (NON RESIDENTIAL)', Arrest=False, Beat=122, District=1, Ward=42, Community Area=32, Year=2023, Latitude=41.886018055, Longitude=-87.633937881, Day=datetime.date(2023, 9, 6), Month=9, WeekDay=4),
 Row(Date='08/31/2023 12:00:00 PM', Block='023XX W JACKSON BLVD', Primary Type='DECEPTIVE PRACTICE', Location Description='STREET', Arrest=False, Beat=1225, District=12, Ward=27, Community Area=28, Year=2023, Latitude=41.877565108, Longitude=-87.68479102, Day=datetime.date(2023, 8, 31), Month=8, WeekDay=5),
 Row(Date='07/24/2023 09:45:00 PM', Block='073XX S JEFFERY BLVD', Primary Type='CRIMINAL SEXUAL ASSAULT', Location Description='APARTMENT', Arrest=False, Beat=333, District=3, Ward=7, Community Area=43, Year=2023, Latitude=41.7619185, Longitude=-87.576209245, Day=datetime.date(2023, 7, 24), Month=7, WeekDay=2),
 Row(Date='08/27/2023 07:00:00 AM', Block='034

In [24]:
dataset.count()

3790328

In [25]:
dataset=dataset.na.drop()

In [26]:
dataset.repartition(1).write.csv('pre_processed_data',mode="append",header=True)

In [31]:
# Filter the dataset to select records after 2018
filtered_dataset = dataset.filter(dataset['Year'] >= 2019)

# Repartition the filtered dataset into a single partition
repartitioned_dataset = filtered_dataset.coalesce(1)

# Write the repartitioned data to a single CSV file
repartitioned_dataset.write.csv('2018onwards', mode="append", header=True)
